In [78]:
import pandas as pd
import numpy as np
import gspread
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
from pandas import json_normalize

import re

import sys                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
sys.path.append('../')

import zoho_crm_api_module as crm

import sys
sys.path.append('../../../zoho_data_app/')

# importing
import main_module as m

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1ZlVGiUYy6mg8R72l2v6VsuApNTvezCTpYDT2JZCXXhs'
SHEET_NAME_KLOM = 'KLOM Wedding Brochure'
SHEET_NAME_KSH = 'KSH Wedding Brochure'


cols = [
    "id",
    "Created_Time",
    "Campaign_Name",
    "First_Name",
    "Last_Name",
    "Email",
    "Phone",
    "Street",
    "City",
    "Country",
    "Brand",
    "Lead_Sub_Brand",
    "Lead_Source",
    "Lead_Source_Description",
    "Lead_Locations",
    "Lead_History",
    "Website"
]

live_cols = [
    "Record Id",
    "Created Time",
    "utm_campaign",
    "First Name",
    "Last Name",
    "Email",
    "Phone",
    "Street",
    "City",
    "Country",
    "Brand",
    "Lead Sub-Brand",
    "Lead Source",
    "Lead Source Description",
    "Lead Locations",
    "utm_source",
    "Website"
]

# Function to fix the encoding
def fix_encoding(garbled_string):
    try:
        # Encode the garbled string to bytes using 'latin1' and then decode it to 'utf-8'
        return garbled_string.encode('latin1').decode('utf-8')
    except Exception as e:
        return garbled_string  # Return the original string if there's an error


In [80]:
crm_token = crm.get_crm_token()

In [81]:
df = crm.get_fb_ads_wedding_data(crm_token, cols)
df.fillna('empty', inplace=True)
new_df = crm.get_fb_ads_wedding_filter(df)
new_df['id'] = 'zcrm_'+new_df['id']
new_df['Created_Time'] = new_df['Created_Time'].str.replace('T', ' ', regex=True)
new_df['Created_Time']  = pd.to_datetime(new_df['Created_Time']).dt.strftime('%d/%m/%Y %H:%M:%S')
new_df['First_Name'] = new_df['First_Name'].apply(fix_encoding)
new_df['Last_Name'] = new_df['Last_Name'].apply(fix_encoding)
m.clean_number(new_df, 'Phone')
m.replace_not_valid_to_empty(new_df, 'Phone')
# new_df = df.copy()
new_df.sort_values('Created_Time', ascending=False, inplace=True)
new_df.reset_index(drop=True, inplace=True)
count_data_awal = new_df.shape[0]
new_df

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\..\zoho_crm_api_module.py:332: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  concat_df['Lead_Locations'].replace('\[|\]|\'','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pand

,id,Created_Time,Campaign_Name,First_Name,Last_Name,Email,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,empty,Claire Anne Adams,casadams@hotmail.co.uk,447838884580,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
1,zcrm_3584690000525549111,25/02/2025 03:08:45,KLOM - Wedding - Leads campaign (Feb 15 2025),empty,Aimee Burns,aimeeburns10@hotmail.com,447943220536,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
2,zcrm_3584690000541349481,24/04/2025 18:52:45,KSM - Leads - Spring Offer - Mar 2025,Rajesh Kumar,Rajesh Kumar,rajesh.kumar@karmagroup.com,918247278724,empty,empty,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
3,zcrm_3584690000541401457,24/04/2025 17:06:12,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,918247278724,empty,empty,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall - Weddings at Karma Salford...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
4,zcrm_3584690000541402416,24/04/2025 16:54:27,KSM - Leads - Spring Offer - Mar 2025,Rajesh Kumar,Rajesh Kumar,rajesh.kumar@karmagroup.com,919525452917,empty,empty,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
5,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,empty,Angela Reilly,crchick0320@yahoo.com,447849972640,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000534473174,24/03/2025 15:55:00,KLOM - Wedding - Leads campaign (Feb 15 2025),empty,Chloe Anderson,chloeroseanderson@hotmail.co.uk,447525298040,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000533729177,24/03/2025 02:28:53,KLOM - Wedding - Leads campaign (Feb 15 2025),empty,Suzanne yeats,suziescot36@gmail.com,447359364074,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
8,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,empty,Emilie Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,empty,empty,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
9,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,empty,Siobhan Mackay,siobhanmackay31@yahoo.co.uk,447707983372,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


In [82]:
new_df['First_Name'] = new_df['First_Name'].str.title()
new_df['Last_Name'] = new_df['Last_Name'].str.title()
new_df['Email'] = m.lowercase(new_df, 'Email')
m.clean_space(new_df, 'Email')

new_df
# new_df.to_excel(r"C:\Users\fajar.fatoni\Documents\Python\Data\kbbvip_live.xlsx", index=False)

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


,id,Created_Time,Campaign_Name,First_Name,Last_Name,Email,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,Empty,Claire Anne Adams,casadams@hotmail.co.uk,447838884580,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
1,zcrm_3584690000525549111,25/02/2025 03:08:45,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Aimee Burns,aimeeburns10@hotmail.com,447943220536,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
2,zcrm_3584690000541349481,24/04/2025 18:52:45,KSM - Leads - Spring Offer - Mar 2025,Rajesh Kumar,Rajesh Kumar,rajesh.kumar@karmagroup.com,918247278724,empty,empty,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
3,zcrm_3584690000541401457,24/04/2025 17:06:12,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,918247278724,empty,empty,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall - Weddings at Karma Salford...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
4,zcrm_3584690000541402416,24/04/2025 16:54:27,KSM - Leads - Spring Offer - Mar 2025,Rajesh Kumar,Rajesh Kumar,rajesh.kumar@karmagroup.com,919525452917,empty,empty,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
5,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,Empty,Angela Reilly,crchick0320@yahoo.com,447849972640,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000534473174,24/03/2025 15:55:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Chloe Anderson,chloeroseanderson@hotmail.co.uk,447525298040,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000533729177,24/03/2025 02:28:53,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Suzanne Yeats,suziescot36@gmail.com,447359364074,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
8,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,Empty,Emilie Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,empty,empty,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
9,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,Empty,Siobhan Mackay,siobhanmackay31@yahoo.co.uk,447707983372,empty,empty,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


In [83]:
# # mencari tanggal 8 hari kebelakang
# calculate_date = datetime.now() - timedelta(days=8)
# new_cal_date = str(calculate_date.strftime("%Y-%m-%d")).split(' ')
# first_date = new_cal_date[0]

# # mencari tanggal hari ini
# today = datetime.now() 
# second_date = today.strftime("%Y-%m-%d")

# print(first_date)
# print(second_date)

In [84]:
new_df.replace('empty', '', inplace=True)
new_df['First_Name'] = new_df['First_Name'].str.title()
new_df['Last_Name'] = new_df['Last_Name'].str.title()
new_df['Email'] = m.lowercase(new_df, 'Email')
m.clean_space(new_df, 'Email')

df_klom = new_df.loc[new_df['Lead_Locations'].str.contains('Karma Lake of Menteith', regex=True, flags=re.I)].copy()
df_ksh = new_df.loc[new_df['Lead_Locations'].str.contains('Karma Salford Hall', regex=True, flags=re.I)].copy()
df_klom

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


,id,Created_Time,Campaign_Name,First_Name,Last_Name,Email,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,Empty,Claire Anne Adams,casadams@hotmail.co.uk,447838884580,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
1,zcrm_3584690000525549111,25/02/2025 03:08:45,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Aimee Burns,aimeeburns10@hotmail.com,447943220536,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
5,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,Empty,Angela Reilly,crchick0320@yahoo.com,447849972640,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000534473174,24/03/2025 15:55:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Chloe Anderson,chloeroseanderson@hotmail.co.uk,447525298040,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000533729177,24/03/2025 02:28:53,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Suzanne Yeats,suziescot36@gmail.com,447359364074,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
8,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,Empty,Emilie Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,,,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
9,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,Empty,Siobhan Mackay,siobhanmackay31@yahoo.co.uk,447707983372,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
10,zcrm_3584690000540832897,22/04/2025 17:00:05,KLOM - Wedding - Leads campaign 2025,Empty,Alison Reynolds,alireynolds67@gmail.com,447979103109,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Elopement We...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
11,zcrm_3584690000533606042,22/03/2025 00:20:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Danielle Anderson,missdanielleee@hotmail.co.uk,447958161492,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
12,zcrm_3584690000525468001,22/02/2025 17:52:46,KLOM - Wedding - Leads campaign (Feb 15 2025),Empty,Jenny Wainwright,jennywainwright1986@outlook.com,447960005139,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


In [85]:
if not df_klom.empty:

    if df_klom['Last_Name'].str.contains(' ', regex=True, flags=re.I).any():
        split_name_klom = df_klom['Last_Name'].str.split(' ', n=1, expand=True)
        df_klom['First_Name'] = split_name_klom[0]
        df_klom['Last_Name'] = split_name_klom[1]
    else:
        df_klom['First_Name'] = df_klom['Last_Name']
        df_klom['Last_Name'] = ''
    display(df_klom)

if not df_ksh.empty:

    if df_ksh['Last_Name'].str.contains(' ', regex=True, flags=re.I).any():
        split_name_ksh = df_ksh['Last_Name'].str.split(' ', n=1, expand=True)
        df_ksh['First_Name'] = split_name_ksh[0]
        df_ksh['Last_Name'] = split_name_ksh[1]
    else:
        df_ksh['First_Name'] = df_ksh['Last_Name']
        df_ksh['Last_Name'] = ''
    display(df_ksh)

,id,Created_Time,Campaign_Name,First_Name,Last_Name,Email,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,Claire,Anne Adams,casadams@hotmail.co.uk,447838884580,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
1,zcrm_3584690000525549111,25/02/2025 03:08:45,KLOM - Wedding - Leads campaign (Feb 15 2025),Aimee,Burns,aimeeburns10@hotmail.com,447943220536,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
5,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,Angela,Reilly,crchick0320@yahoo.com,447849972640,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000534473174,24/03/2025 15:55:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Chloe,Anderson,chloeroseanderson@hotmail.co.uk,447525298040,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000533729177,24/03/2025 02:28:53,KLOM - Wedding - Leads campaign (Feb 15 2025),Suzanne,Yeats,suziescot36@gmail.com,447359364074,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
8,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,Emilie,Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,,,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
9,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,Siobhan,Mackay,siobhanmackay31@yahoo.co.uk,447707983372,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
10,zcrm_3584690000540832897,22/04/2025 17:00:05,KLOM - Wedding - Leads campaign 2025,Alison,Reynolds,alireynolds67@gmail.com,447979103109,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Elopement We...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
11,zcrm_3584690000533606042,22/03/2025 00:20:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Danielle,Anderson,missdanielleee@hotmail.co.uk,447958161492,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
12,zcrm_3584690000525468001,22/02/2025 17:52:46,KLOM - Wedding - Leads campaign (Feb 15 2025),Jenny,Wainwright,jennywainwright1986@outlook.com,447960005139,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


,id,Created_Time,Campaign_Name,First_Name,Last_Name,Email,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website
2,zcrm_3584690000541349481,24/04/2025 18:52:45,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
3,zcrm_3584690000541401457,24/04/2025 17:06:12,KSM - Leads - Spring Offer - Mar 2025,Kumar,None,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall - Weddings at Karma Salford...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
4,zcrm_3584690000541402416,24/04/2025 16:54:27,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919525452917,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
15,zcrm_3584690000525236001,20/02/2025 19:22:48,KLM - Wedding - Leads campaign (Jan 2025),Rajesh,Kumar,rajesh.kumar@karmagroup.com,919009887776,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
20,zcrm_3584690000530900534,17/03/2025 16:15:11,KB - Lead - Flash Sale (feb 28 -mar 1) 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919987654433,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...


In [86]:
df_live_klom = pd.DataFrame() # Initialize an empty DataFrame for KLOM
df_live_ksh = pd.DataFrame() # Initialize an empty DataFrame for KSH

try:
    # Get data from Google Sheets
    spreadsheet = gc.open_by_key(SHEET_ID)

    # --- Process KLOM Sheet ---
    try:
        worksheet_klom = spreadsheet.worksheet(SHEET_NAME_KLOM)
        # get_all_records assumes the first row is a header
        rows_klom = worksheet_klom.get_all_records()

        # Check if any data was returned for KLOM
        if not rows_klom:
            print(f"No data found in sheet '{SHEET_NAME_KLOM}' or it lacks a header row.")
            # Define expected columns for an empty DataFrame if needed downstream
            df_live_klom = pd.DataFrame(columns=['First Name', 'Last Name'])
        else:
            # Create DataFrame from the rows
            df_live_klom = pd.DataFrame(rows_klom)

            # --- Process columns for KLOM only if they exist ---

            # Process 'First Name' for KLOM
            if 'First Name' in df_live_klom.columns:
                df_live_klom['First Name'] = df_live_klom['First Name'].apply(fix_encoding)
            else:
                print("Warning (KLOM): Column 'First Name' not found in the sheet.")

            # Process 'Last Name' for KLOM
            if 'Last Name' in df_live_klom.columns:
                df_live_klom['Last Name'] = df_live_klom['Last Name'].apply(fix_encoding)
            else:
                print("Warning (KLOM): Column 'Last Name' not found in the sheet.")

    except Exception as e_klom:
        print(f"An error occurred while accessing or processing KLOM sheet: {e_klom}")
        # Ensure df_live_klom is an empty DataFrame in case of error during processing
        if not isinstance(df_live_klom, pd.DataFrame) or df_live_klom.empty:
            df_live_klom = pd.DataFrame(columns=['First Name', 'Last Name']) # Or just pd.DataFrame()
        print("KLOM DataFrame might be empty or partially processed due to the error.")

    # --- Process KSH Sheet ---
    try:
        worksheet_ksh = spreadsheet.worksheet(SHEET_NAME_KSH)
        # get_all_records assumes the first row is a header
        rows_ksh = worksheet_ksh.get_all_records()

        # Check if any data was returned for KSH
        if not rows_ksh:
            print(f"No data found in sheet '{SHEET_NAME_KSH}' or it lacks a header row.")
            # Define expected columns for an empty DataFrame if needed downstream
            df_live_ksh = pd.DataFrame() # Initialize with no specific columns if structure is unknown
        else:
            # Create DataFrame from the rows
            df_live_ksh = pd.DataFrame(rows_ksh)

            # --- Process columns for KSH (add your specific column processing here) ---
            # Example: Assuming KSH also has 'Some_Column' that needs encoding
            if 'Some_Column' in df_live_ksh.columns:
                df_live_ksh['Some_Column'] = df_live_ksh['Some_Column'].apply(fix_encoding)
            else:
                print("Warning (KSH): Column 'Some_Column' not found in the sheet.")

            # Add more column processing for df_live_ksh as needed

    except Exception as e_ksh:
        print(f"An error occurred while accessing or processing KSH sheet: {e_ksh}")
        # Ensure df_live_ksh is an empty DataFrame in case of error during processing
        if not isinstance(df_live_ksh, pd.DataFrame) or df_live_ksh.empty:
            df_live_ksh = pd.DataFrame() # Or define columns if known
        print("KSH DataFrame might be empty or partially processed due to the error.")

except Exception as e_overall:
    print(f"An overall error occurred: {e_overall}")
    # Ensure both DataFrames are empty in case of a fundamental error
    df_live_klom = pd.DataFrame(columns=['First Name', 'Last Name'])
    df_live_ksh = pd.DataFrame()
    print("Both DataFrames might be empty due to the overall error.")

# Display the DataFrames
print("\nFinal DataFrame contents for KLOM:")
try:
    display(df_live_klom)
except NameError:
    print(df_live_klom)

print("\nFinal DataFrame contents for KSH:")
try:
    display(df_live_ksh)
except NameError:
    print(df_live_ksh)

No data found in sheet 'KLOM Wedding Brochure' or it lacks a header row.
Warning (KSH): Column 'Some_Column' not found in the sheet.

Final DataFrame contents for KLOM:


,First Name,Last Name



Final DataFrame contents for KSH:


,Record Id,Created Time,utm_campaign,First Name,Last Name,Email,Phone,Street,City,Country,Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,utm_source,Website
0,zcrm_3584690000541349481,24/04/2025 18:52:45,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
1,zcrm_3584690000541401457,24/04/2025 17:06:12,KSM - Leads - Spring Offer - Mar 2025,Kumar,,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall - Weddings at Karma Salford...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
2,zcrm_3584690000541402416,24/04/2025 16:54:27,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919525452917,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
3,zcrm_3584690000530900534,17/03/2025 16:15:11,KB - Lead - Flash Sale (feb 28 -mar 1) 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919987654433,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
4,zcrm_3584690000525236001,20/02/2025 19:22:48,KLM - Wedding - Leads campaign (Jan 2025),Rajesh,Kumar,rajesh.kumar@karmagroup.com,919009887776,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...


In [87]:
if not df_live_klom.empty:
    df_update_klom = df_klom.loc[~df_klom['id'].isin(df_live_klom['Record Id'])].copy()
else :
    df_update_klom = df_klom.copy()

if not df_live_ksh.empty:
    df_update_ksh = df_ksh.loc[~df_ksh['id'].isin(df_live_ksh['Record Id'])].copy()
else :
    df_update_ksh = df_ksh.copy()

display(df_update_klom)
display(df_update_ksh)

,id,Created_Time,Campaign_Name,First_Name,Last_Name,Email,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,Claire,Anne Adams,casadams@hotmail.co.uk,447838884580,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
1,zcrm_3584690000525549111,25/02/2025 03:08:45,KLOM - Wedding - Leads campaign (Feb 15 2025),Aimee,Burns,aimeeburns10@hotmail.com,447943220536,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
5,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,Angela,Reilly,crchick0320@yahoo.com,447849972640,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000534473174,24/03/2025 15:55:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Chloe,Anderson,chloeroseanderson@hotmail.co.uk,447525298040,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000533729177,24/03/2025 02:28:53,KLOM - Wedding - Leads campaign (Feb 15 2025),Suzanne,Yeats,suziescot36@gmail.com,447359364074,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
8,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,Emilie,Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,,,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
9,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,Siobhan,Mackay,siobhanmackay31@yahoo.co.uk,447707983372,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
10,zcrm_3584690000540832897,22/04/2025 17:00:05,KLOM - Wedding - Leads campaign 2025,Alison,Reynolds,alireynolds67@gmail.com,447979103109,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Elopement We...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
11,zcrm_3584690000533606042,22/03/2025 00:20:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Danielle,Anderson,missdanielleee@hotmail.co.uk,447958161492,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
12,zcrm_3584690000525468001,22/02/2025 17:52:46,KLOM - Wedding - Leads campaign (Feb 15 2025),Jenny,Wainwright,jennywainwright1986@outlook.com,447960005139,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


,id,Created_Time,Campaign_Name,First_Name,Last_Name,Email,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website


In [88]:
df_update_klom.sort_values(by=['Created_Time'], inplace=True, ascending=False)
rename_mapping = dict(zip(cols, live_cols))
df_update_klom = df_update_klom.rename(columns=rename_mapping)
df_update_klom.reset_index(drop=True, inplace=True)

df_update_ksh.sort_values(by=['Created_Time'], inplace=True, ascending=False)
rename_mapping = dict(zip(cols, live_cols))
df_update_ksh = df_update_ksh.rename(columns=rename_mapping)
df_update_ksh.reset_index(drop=True, inplace=True)

display(df_update_klom)
display(df_update_ksh)


,Record Id,Created Time,utm_campaign,First Name,Last Name,Email,Phone,Street,City,Country,Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,utm_source,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,Claire,Anne Adams,casadams@hotmail.co.uk,447838884580,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
1,zcrm_3584690000525549111,25/02/2025 03:08:45,KLOM - Wedding - Leads campaign (Feb 15 2025),Aimee,Burns,aimeeburns10@hotmail.com,447943220536,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
2,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,Angela,Reilly,crchick0320@yahoo.com,447849972640,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
3,zcrm_3584690000534473174,24/03/2025 15:55:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Chloe,Anderson,chloeroseanderson@hotmail.co.uk,447525298040,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
4,zcrm_3584690000533729177,24/03/2025 02:28:53,KLOM - Wedding - Leads campaign (Feb 15 2025),Suzanne,Yeats,suziescot36@gmail.com,447359364074,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
5,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,Emilie,Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,,,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,Siobhan,Mackay,siobhanmackay31@yahoo.co.uk,447707983372,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000540832897,22/04/2025 17:00:05,KLOM - Wedding - Leads campaign 2025,Alison,Reynolds,alireynolds67@gmail.com,447979103109,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Elopement We...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
8,zcrm_3584690000533606042,22/03/2025 00:20:00,KLOM - Wedding - Leads campaign (Feb 15 2025),Danielle,Anderson,missdanielleee@hotmail.co.uk,447958161492,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
9,zcrm_3584690000525468001,22/02/2025 17:52:46,KLOM - Wedding - Leads campaign (Feb 15 2025),Jenny,Wainwright,jennywainwright1986@outlook.com,447960005139,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


,Record Id,Created Time,utm_campaign,First Name,Last Name,Email,Phone,Street,City,Country,Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,utm_source,Website


In [89]:
# Concatenate DataFrames
df_final_klom = pd.concat([df_update_klom, df_live_klom], ignore_index=True)
df_final_ksh = pd.concat([df_update_ksh, df_live_ksh], ignore_index=True)

# Replace 'empty' with ''
df_final_klom.replace('empty', '', inplace=True)
df_final_ksh.replace('empty', '', inplace=True)

# Convert 'Created Time' to datetime format
df_final_klom['Created Time'] = pd.to_datetime(df_final_klom['Created Time'], dayfirst=True, errors='coerce')
df_final_ksh['Created Time'] = pd.to_datetime(df_final_ksh['Created Time'], dayfirst=True, errors='coerce')

# Sort by 'Created Time' in descending order
df_final_klom.sort_values('Created Time', ascending=False, inplace=True)
df_final_ksh.sort_values('Created Time', ascending=False, inplace=True)

# Format 'Created Time' back to the desired string format
df_final_klom['Created Time'] = df_final_klom['Created Time'].dt.strftime('%d/%m/%Y %H:%M:%S')
df_final_ksh['Created Time'] = df_final_ksh['Created Time'].dt.strftime('%d/%m/%Y %H:%M:%S')

display(df_final_klom)
display(df_final_ksh)

,Record Id,Created Time,utm_campaign,First Name,Last Name,Email,Phone,Street,City,Country,Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,utm_source,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,Claire,Anne Adams,casadams@hotmail.co.uk,447838884580,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
2,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,Angela,Reilly,crchick0320@yahoo.com,447849972640,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
5,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,Emilie,Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,,,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,Siobhan,Mackay,siobhanmackay31@yahoo.co.uk,447707983372,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000540832897,22/04/2025 17:00:05,KLOM - Wedding - Leads campaign 2025,Alison,Reynolds,alireynolds67@gmail.com,447979103109,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Elopement We...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
12,zcrm_3584690000540704042,19/04/2025 15:36:53,KLOM - Wedding - Leads campaign 2025,Ailie,Gordon,ailie_ann@hotmail.com,447525161492,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
15,zcrm_3584690000540571078,18/04/2025 05:14:28,KLOM - Wedding - Leads campaign 2025,Jemma,Reid / Grant Binnie,jgbinnie11@gmail.com,447858529899,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
17,zcrm_3584690000540426001,16/04/2025 17:20:14,KLOM - Wedding - Leads campaign 2025,Suzanne,Yeats,suziescot36@gmail.com,447359364074,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
23,zcrm_3584690000540341001,15/04/2025 21:12:38,KLOM - Wedding - Leads campaign 2025,Sarah-Jo,Douglas,ybnlhq@live.com,447713355810,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
25,zcrm_3584690000537891106,13/04/2025 02:26:14,KLOM - Wedding - Leads campaign 2025,Emma,Taylor,emmataylorrrrr@gmail.com,447968241355,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


,Record Id,Created Time,utm_campaign,First Name,Last Name,Email,Phone,Street,City,Country,Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,utm_source,Website
0,zcrm_3584690000541349481,24/04/2025 18:52:45,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
1,zcrm_3584690000541401457,24/04/2025 17:06:12,KSM - Leads - Spring Offer - Mar 2025,Kumar,,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall - Weddings at Karma Salford...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
2,zcrm_3584690000541402416,24/04/2025 16:54:27,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919525452917,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
3,zcrm_3584690000530900534,17/03/2025 16:15:11,KB - Lead - Flash Sale (feb 28 -mar 1) 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919987654433,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
4,zcrm_3584690000525236001,20/02/2025 19:22:48,KLM - Wedding - Leads campaign (Jan 2025),Rajesh,Kumar,rajesh.kumar@karmagroup.com,919009887776,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...


In [90]:
# Function to sanitize DataFrame
def sanitize_dataframe(df):
    # Replace invalid float values with None
    df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)
    return df

# Sanitize the DataFrame
df_final_klom = sanitize_dataframe(df_final_klom)
df_final_klom.drop(df_final_klom.loc[df_final_klom['Email'].str.contains('karmagroup.com', regex=True, flags=re.I)].index, inplace=True)
display(df_final_klom)

# Sanitize the DataFrame
df_final_ksh = sanitize_dataframe(df_final_ksh)
# df_final_ksh.drop(df_final_ksh.loc[df_final_ksh['Email'].str.contains('karmagroup.com', regex=True, flags=re.I)].index, inplace=True)
display(df_final_ksh)

C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_8948\3825288471.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)


,Record Id,Created Time,utm_campaign,First Name,Last Name,Email,Phone,Street,City,Country,Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,utm_source,Website
0,zcrm_3584690000541475066,25/04/2025 10:01:36,KLOM - Wedding - Leads campaign 2025,Claire,Anne Adams,casadams@hotmail.co.uk,447838884580,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
2,zcrm_3584690000541295066,24/04/2025 00:24:10,KLOM - Wedding - Leads campaign 2025,Angela,Reilly,crchick0320@yahoo.com,447849972640,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
5,zcrm_3584690000541295033,23/04/2025 23:16:07,KLOM - Wedding - Leads campaign 2025,Emilie,Wojtas Olorenshaw,ellyterre@hotmail.com,33681102767,,,France,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
6,zcrm_3584690000541096002,23/04/2025 03:17:06,KLOM - Wedding - Leads campaign 2025,Siobhan,Mackay,siobhanmackay31@yahoo.co.uk,447707983372,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
7,zcrm_3584690000540832897,22/04/2025 17:00:05,KLOM - Wedding - Leads campaign 2025,Alison,Reynolds,alireynolds67@gmail.com,447979103109,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Elopement We...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
12,zcrm_3584690000540704042,19/04/2025 15:36:53,KLOM - Wedding - Leads campaign 2025,Ailie,Gordon,ailie_ann@hotmail.com,447525161492,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
15,zcrm_3584690000540571078,18/04/2025 05:14:28,KLOM - Wedding - Leads campaign 2025,Jemma,Reid / Grant Binnie,jgbinnie11@gmail.com,447858529899,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
17,zcrm_3584690000540426001,16/04/2025 17:20:14,KLOM - Wedding - Leads campaign 2025,Suzanne,Yeats,suziescot36@gmail.com,447359364074,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
23,zcrm_3584690000540341001,15/04/2025 21:12:38,KLOM - Wedding - Leads campaign 2025,Sarah-Jo,Douglas,ybnlhq@live.com,447713355810,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...
25,zcrm_3584690000537891106,13/04/2025 02:26:14,KLOM - Wedding - Leads campaign 2025,Emma,Taylor,emmataylorrrrr@gmail.com,447968241355,,,United Kingdom,Karma Resorts,Other,Web Enquiry,Karma Lake Of Menteith - Download Wedding Broc...,Karma Lake Of Menteith,meta,https://karmagroup.com/weddings-at-karma-lake-...


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_8948\3825288471.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)


,Record Id,Created Time,utm_campaign,First Name,Last Name,Email,Phone,Street,City,Country,Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,utm_source,Website
0,zcrm_3584690000541349481,24/04/2025 18:52:45,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
1,zcrm_3584690000541401457,24/04/2025 17:06:12,KSM - Leads - Spring Offer - Mar 2025,Kumar,,rajesh.kumar@karmagroup.com,918247278724,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall - Weddings at Karma Salford...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
2,zcrm_3584690000541402416,24/04/2025 16:54:27,KSM - Leads - Spring Offer - Mar 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919525452917,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
3,zcrm_3584690000530900534,17/03/2025 16:15:11,KB - Lead - Flash Sale (feb 28 -mar 1) 2025,Rajesh,Kumar,rajesh.kumar@karmagroup.com,919987654433,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...
4,zcrm_3584690000525236001,20/02/2025 19:22:48,KLM - Wedding - Leads campaign (Jan 2025),Rajesh,Kumar,rajesh.kumar@karmagroup.com,919009887776,,,India,Karma Resorts,Other,Web Enquiry,Karma Salford Hall -Download Wedding Brochure ...,Karma Salford Hall,meta,https://karmagroup.com/weddings-at-karma-salfo...


In [91]:

if not df_final_klom.empty:
    # Update keseluruhan data, delete data yang lama terlebih dahulu
    # clear all row before update
    worksheet_klom.clear()
    worksheet_klom.update([df_final_klom.columns.values.tolist()] + df_final_klom.values.tolist())

if not df_final_ksh.empty:
    # Update keseluruhan data, delete data yang lama terlebih dahulu
    # clear all row before update
    worksheet_ksh.clear()
    worksheet_ksh.update([df_final_ksh.columns.values.tolist()] + df_final_ksh.values.tolist())